In [3]:
import os
import sys
sys.path.append("./ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
from utils.bernstein import bernstein_coeff_order10_new
from ddn.ddn.pytorch.node import AbstractDeclarativeNode
from utils.models.ddn import *
from utils.nodes.OPTNode import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
rho_obs = 1.2
rho_eq = 10.0
weight_smoothness = 10

pred_len = 30
num = 30
t_fin = 2.0
a_obs = 1.0
b_obs = 1.0

tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]

x_obs_temp = np.hstack((-10000.0, 10000.79, 30000.0, 10000.0))
y_obs_temp = np.hstack((-10000.0, 10000.0, -30000.80, 10000.0))

num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]

obs_inp_list = []
for i in range(num_obs):
    obs_inp_list.extend([x_obs_temp[i], y_obs_temp[i], a_obs])
    
A_obs = np.tile(P, (num_obs, 1))
A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
Q_smoothness = np.dot(Pddot.T, Pddot)

In [ ]:
problem = OPTNode(rho_eq=10, t_fin=9.0, num=pred_len, bernstein_coeff_order10_new=bernstein_coeff_order10_new, device = "cpu")
opt_layer = DeclarativeLayer(problem)

In [ ]:
x_init, y_init, v_init, psi_init, psidot_init = 0.0, 0.0, 0.0, 0.0, 0.0
x_fin, y_fin, psi_fin = 10.0, 10.0, 0.0
# sol = problem.solve(fixed_params, variable_params)

In [ ]:
fixed_params = torch.tensor([x_init, y_init, v_init, psi_init, psidot_init], dtype=torch.double).reshape(1, 5)
variable_params = torch.tensor([x_fin, y_fin, psi_fin], dtype=torch.double).reshape(1, 3)
sol = problem.solve(fixed_params, variable_params)

In [ ]:
class toy(nn.Module):
    def __init__(self, opt_layer):
        super(toy, self).__init__()
        self.opt_layer = opt_layer
        self.activation = nn.ReLU()
        self.layer = nn.Linear(63, 63)
        # self.mask = torch.tensor([[0.0, 0.0, 1.0]], dtype=torch.double)
    
    def forward(self, fixed_params, variable_params):
        return self.activation(self.layer(self.opt_layer(fixed_params, variable_params)))
    
model = toy(opt_layer)

In [11]:
model.double()
out = model(fixed_params, variable_params)
loss = nn.MSELoss()
criterion = loss(out, torch.rand(1, 63, dtype=torch.double))
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
optimizer.zero_grad()
criterion.backward()
optimizer.step()

In [15]:
!pip3 install torchviz
from torchviz import make_dot
make_dot(out, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

'rnn_torchviz.png'

In [16]:
print()
print('Tracing back tensors:')
def getBack(var_grad_fn):
    print(var_grad_fn)
    for n in var_grad_fn.next_functions:
        if n[0]:
            try:
                tensor = getattr(n[0], 'variable')
                print(n[0])
                print('Tensor with grad found:', tensor)
                print(' - gradient:', tensor.grad)
                print()
            except AttributeError as e:
                getBack(n[0])

getBack(criterion.grad_fn)


Tracing back tensors:
Tensor with grad found: Parameter containing:
tensor([ 0.0438, -0.1053, -0.0841, -0.0398,  0.0519, -0.1223,  0.0986,  0.0224,
        -0.0734,  0.0217,  0.0735, -0.0200,  0.0392,  0.0472,  0.0986, -0.0484,
        -0.1041,  0.0016,  0.0236, -0.0003,  0.0485,  0.0331, -0.0179,  0.0413,
        -0.1136, -0.0300,  0.0538, -0.0665,  0.1141, -0.0791, -0.1090,  0.0496,
        -0.0140, -0.1150,  0.0695,  0.0011,  0.0587, -0.0418, -0.0989,  0.0271,
         0.0673, -0.0161, -0.0187, -0.1251, -0.0267, -0.0437, -0.1186,  0.0491,
         0.1027, -0.0625, -0.0443, -0.0867,  0.0442, -0.0863, -0.0922, -0.0005,
        -0.0117, -0.0655,  0.0379, -0.1023,  0.0304,  0.1150, -0.1227],
       dtype=torch.float64, requires_grad=True)
 - gradient: tensor([ 0.0000,  0.0000,  0.0554,  0.0000,  0.0530,  0.0463,  0.1020,  0.0000,
        -0.0143, -0.0052,  0.0000,  0.0211,  0.0000,  0.0206,  0.0000, -0.0119,
        -0.0173,  0.0674,  0.0000,  0.0046,  0.0000,  0.0000,  0.0000, -0.0058